# Réconciliation des Séries Temporelles : Approche Bottom-Up

La réconciliation des séries temporelles est un processus visant à garantir que les prévisions ou observations à différents niveaux d'agrégation respectent une structure hiérarchique ou de groupe cohérente. Cela est particulièrement utile lorsque vous avez des prévisions individuelles pour plusieurs entités (par exemple, des magasins, des régions ou des produits) qui doivent être consolidées pour respecter des totaux au niveau global.

## Approche Bottom-Up

Dans l'approche **bottom-up**, les prévisions sont d'abord réalisées pour les niveaux les plus détaillés, puis agrégées pour obtenir des estimations aux niveaux supérieurs. Par exemple, pour vos 5 clusters de magasins, les prévisions individuelles sont simplement additionnées pour obtenir une estimation globale :

$$\hat{y}_{global} = \sum_{i=1}^{n} \hat{y}_{cluster_i}$$

## Étapes clés

1. **Préparer les données** 

2. **Réaliser les prévisions** :

   -> Appliquer un modèle de prévision pour chaque cluster (SARIMA, Prophet Online).

3. **Effectuer l'agrégation** 

4. **Valider les résultats** 



## -> Limites
- **Biais potentiel** : Les erreurs des prévisions au niveau inférieur se propagent et s'accumulent au niveau global.
- **Manque de flexibilité** : Cette approche ne corrige pas automatiquement les incohérences entre les niveaux.

La réconciliation vise à regler ces problèmes. 


### Réconciliation des Prédictions SARIMA par Clusters

Dans cette section, nous allons réconcilier les prédictions des modèles SARIMA sélectionnés par la fonction `auto_arima` pour chaque cluster. La réconciliation sera effectuée de manière proportionnelle au sein de chaque cluster. 

L'objectif est d'évaluer si cette approche permet d'améliorer les prédictions pour chaque cluster de manière individuelle.

In [114]:
import numpy as np
import pandas as pd
import plotly.express as px
from functions import *
import plotly.graph_objects as go

### 📥 Importation et Prétraitement des Données

In [115]:
Cluster_0 = pd.read_csv('Cluster_0.csv')
Cluster_1 = pd.read_csv('Cluster_1.csv')
Cluster_2 = pd.read_csv('Cluster_2.csv')
Cluster_3 = pd.read_csv('Cluster_3.csv')
Cluster_4 = pd.read_csv('Cluster_4.csv')
test_clustered = pd.read_csv("all_clusters_time_series_test.csv")
All_prevision = pd.read_csv('All_prevision.csv')
All_prevision.head()

,date,sales,Test_Forecast,test_Forecast_prophet_online,test_Forecast_prophet_online_global,Test_Forecast_Sarimax_global,Test_Forecast_prophet_online
0,2016-01-01,1.643339e+04,923070.383856,56233.784306,690101.175402,8.832907e+05,692107.112131
1,2016-01-02,1.066677e+06,906475.369087,70606.134717,869241.978477,6.229928e+05,871434.184993
2,2016-01-03,1.226736e+06,845104.398780,80512.550636,918947.551836,1.115351e+06,921342.024227
3,2016-01-04,9.559569e+05,951790.485532,55072.849906,670285.418443,9.056684e+05,672867.408305
4,2016-01-05,8.353204e+05,884962.934882,55272.762496,637981.921649,9.304216e+05,640742.555912


In [116]:
#--ajustement :  on prend les valeurs de prevision pour chaque cluster dumodel sarimax chosi par les auto_arima--
Cluster_0['Test_Forecast'] = Cluster_0['Test Forecast']
Cluster_1['Test_Forecast'] = Cluster_1['Test Forecast']
Cluster_2['Test_Forecast'] = Cluster_2['Test Forecast']
Cluster_3['Test_Forecast'] = Cluster_3['Test Forecast']
Cluster_4['Test_Forecast'] = Cluster_4['Test Forecast']

### 📊 Définitions des variables

In [117]:
y_global = np.array(All_prevision['Test_Forecast_Sarimax_global'])

y_clusters = np.array([Cluster_0['Test_Forecast'],Cluster_1['Test_Forecast'],Cluster_2['Test_Forecast'],Cluster_3['Test_Forecast'],Cluster_4['Test_Forecast']])

y_clusters_sum = np.sum(y_clusters, axis=0)

$ \Delta = y_{\text{global}} - y_{\text{clusters\_sum}} $  

In [118]:
delta = y_global - y_clusters_sum

$ y_{\text{clusters\_adjusted}} = y_{\text{clusters}} + \left( \Delta \cdot \frac{y_{\text{clusters}}}{y_{\text{clusters\_sum}}} \right) $

In [119]:
y_clusters_adjusted = y_clusters + (delta * (y_clusters / y_clusters_sum))

In [120]:
All_prevision['Test_Forecast_reconciliation'] = [sum(sublist[i] for sublist in y_clusters_adjusted) for i in range(len(y_clusters_adjusted[0]))]

In [121]:
fig = px.line(All_prevision, x='date', y=['sales', 'Test_Forecast','Test_Forecast_Sarimax_global','Test_Forecast_reconciliation'],title="Predictions du test Sarima")
fig.show()

Aprés réconciliation on a $
\text{Prévision globale} = \sum \text{Prévisions des clusters}
$

In [122]:
test_clustered.loc[test_clustered['cluster'] == 'Cluster 0']['sales']

0           0.000000
1       86627.277016
2      110534.310890
3       74601.247117
4       68187.265000
           ...      
603         0.000000
604         0.000000
605         0.000000
606         0.000000
607         0.000000
Name: sales, Length: 608, dtype: float64

In [123]:
df = pd.DataFrame(y_clusters_adjusted).T
df['date'] = All_prevision['date']
df.columns = ['after reconciliation 0','after reconciliation 1','after reconciliation 2','after reconciliation 3','after reconciliation 4','date']
df['before reconciliation 0'] = Cluster_0['Test_Forecast']
df['before reconciliation 1'] = Cluster_1['Test_Forecast']
df['before reconciliation 2'] = Cluster_2['Test_Forecast']
df['before reconciliation 3'] = Cluster_3['Test_Forecast']
df['before reconciliation 4'] = Cluster_4['Test_Forecast']
df

,after reconciliation 0,after reconciliation 1,after reconciliation 2,after reconciliation 3,after reconciliation 4,date,before reconciliation 0,before reconciliation 1,before reconciliation 2,before reconciliation 3,before reconciliation 4
0,88635.965084,188061.795644,166527.909988,238452.879175,201612.129664,2016-01-01,79084.115496,167795.327247,148582.039518,212756.018611,179885.410315
1,51556.439432,139543.336352,130762.266501,169755.862906,131374.879364,2016-01-02,69999.225206,189460.822642,177538.585714,230480.983724,178370.342644
2,94626.316702,254961.901478,242762.546019,314265.392584,208734.617462,2016-01-03,87532.289577,235847.698284,224562.914601,290705.274246,193086.021069
3,79591.126212,216922.198985,201929.694480,251363.476212,155861.925824,2016-01-04,98110.070532,267394.787023,248913.886647,309849.722627,192127.254176
4,77882.394919,215809.707956,191100.371199,263110.606586,182518.540275,2016-01-05,77741.496533,215419.282885,190754.648217,262634.608642,182188.342831
...,...,...,...,...,...,...,...,...,...,...,...
603,155444.756585,301014.664816,179998.070843,402593.816925,138069.862674,2017-08-27,189985.820727,367902.523074,219995.077157,492053.372586,168750.086875
604,191867.222005,372933.165785,229111.767248,501903.268112,169577.567742,2017-08-28,192338.622697,373849.429389,229674.674473,503136.399782,169994.204736
605,212243.901953,412946.706781,256707.889949,555754.325816,185869.230593,2017-08-29,194193.867844,377828.137652,234876.468047,508490.849950,170062.199525
606,267778.014984,518091.932751,311543.808678,695322.453099,237687.195759,2017-08-30,190903.550394,369356.645647,222104.937119,495707.330466,169451.288061


In [127]:
fig = px.line(df,x='date',y=df.columns,title='Séries temporelles regroupées par cluster reconciliées',)
fig.show()


In [125]:
fig = px.line(
    test_clustered,
    x='date',
    y='sales',
    color='cluster',
    title='Séries temporelles regroupées par cluster',
    labels={'date': 'Date', 'sales': 'Ventes', 'cluster': 'Cluster'}
)

fig.show()

/Users/lilya/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Affichons toutes ces courbes sur le même graphe pour pouvoir comparer.

In [128]:
fig = go.Figure()

for col in df.columns:
    fig.add_trace(go.Scatter(
        x=df['date'],
        y=df[col],
        mode='lines',
        name=col
    ))

for cluster in test_clustered['cluster'].unique():
    cluster_data = test_clustered[test_clustered['cluster'] == cluster]
    fig.add_trace(go.Scatter(
        x=cluster_data['date'],
        y=cluster_data['sales'],
        mode='lines',
        name=cluster
    ))

fig.update_layout(
    title='Séries temporelles regroupées par cluster et réconciliées',
    xaxis_title='Date',
    yaxis_title='Ventes',
    legend_title='Séries'
)
fig.show()

### Analyse des Résultats après Réconciliation

En cliquant sur une combinaison entre **"After Reconciliation $i$"**, **"Before Reconciliation $i$"**, et **"Cluster $i$"**, on observe que la réconciliation a permis d'obtenir une prédiction pour le $i^{\text{ème}}$ cluster plus proche de la réalité (Cluster $i$).